<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/5_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 改用 threading

1. step1-4 直接包成一整個function (含progress)
2. default progres 修改
3. dcc.Interval
4. dcc.Loading
5. 結束後 interval 關閉

In [ ]:
import dash
from dash import html, dcc, Input, Output, State
import threading
import time

app = dash.Dash(__name__)
server = app.server

# 步驟清單
step_names = [
    "撈取資料",
    "確認與篩選資料",
    "資料預處理",
    "進行 Rule Check",
    "繪圖",
    "上拋資料"
]

# 全域進度狀態
progress = {
    'current_index': -1,
    'step_status': ['pending'] * len(step_names),  # pending / running / success / error
    'step_times': [None] * len(step_names),
    'error': False,
    'done': False
}

# 背景執行報告流程
def run_report_process():
    try:
        # Step 1: 撈取資料
        progress['current_index'] = 0
        progress['step_status'][0] = 'running'
        start = time.time()
        time.sleep(1)  # 模擬
        progress['step_status'][0] = 'success'
        progress['step_times'][0] = round(time.time() - start, 2)

        # Step 2: 確認與篩選資料
        progress['current_index'] = 1
        progress['step_status'][1] = 'running'
        start = time.time()
        time.sleep(1)
        progress['step_status'][1] = 'success'
        progress['step_times'][1] = round(time.time() - start, 2)

        # Step 3: 資料預處理
        progress['current_index'] = 2
        progress['step_status'][2] = 'running'
        start = time.time()
        time.sleep(1)
        progress['step_status'][2] = 'success'
        progress['step_times'][2] = round(time.time() - start, 2)

        # Step 4: 進行 Rule Check
        progress['current_index'] = 3
        progress['step_status'][3] = 'running'
        start = time.time()
        time.sleep(1)
        progress['step_status'][3] = 'success'
        progress['step_times'][3] = round(time.time() - start, 2)

        # Step 5: 繪圖
        progress['current_index'] = 4
        progress['step_status'][4] = 'running'
        start = time.time()
        time.sleep(1)
        progress['step_status'][4] = 'success'
        progress['step_times'][4] = round(time.time() - start, 2)

        # Step 6: 上拋資料
        progress['current_index'] = 5
        progress['step_status'][5] = 'running'
        start = time.time()
        time.sleep(1)
        progress['step_status'][5] = 'success'
        progress['step_times'][5] = round(time.time() - start, 2)

        progress['done'] = True

    except Exception as e:
        progress['step_status'][progress['current_index']] = 'error'
        progress['error'] = True


# Layout
app.layout = html.Div([
    html.H2("報告產出流程"),
    html.Button("開始產出報告", id="run-btn", n_clicks=0),
    dcc.Loading(  # <-- 這裡包整個 step-list + final-btn-area
        type="default",
        children=html.Div([
            html.Div(id="step-list", style={"marginTop": "20px", "fontSize": "16px"}),
            html.Div(id="final-btn-area", style={"marginTop": "20px"})
        ])
    ),
    dcc.Interval(id="interval", interval=1000, n_intervals=0, disabled=True)
])

# 按下按鈕後隱藏按鈕並啟動流程
@app.callback(
    Output("interval", "disabled"),
    Output("run-btn", "style"),
    Input("run-btn", "n_clicks"),
    prevent_initial_call=True
)
def start_process(n_clicks):
    # 初始化
    progress['current_index'] = 0
    progress['step_status'] = ['pending'] * len(step_names)
    progress['step_status'][0] = 'running'
    progress['step_times'] = [None] * len(step_names)
    progress['error'] = False
    progress['done'] = False

    thread = threading.Thread(target=run_report_process)
    thread.start()
    return False, {"display": "none"}


# UI 更新
@app.callback(
    Output("step-list", "children"),
    Output("final-btn-area", "children"),
    Output("interval", "disabled"),  # 新增這個 Output
    Input("interval", "n_intervals"),
    prevent_initial_call=True
)
def update_status(n):
    steps_ui = []
    for i, name in enumerate(step_names):
        status = progress['step_status'][i]
        if status == 'pending':
            color = 'gray'
            prefix = '□'
        elif status == 'running':
            color = 'orange'
            prefix = '▶'
        elif status == 'success':
            color = 'green'
            prefix = '✔'
        elif status == 'error':
            color = 'red'
            prefix = '❌'
        else:
            color = 'gray'
            prefix = '□'

        time_str = f" ({progress['step_times'][i]}秒)" if progress['step_times'][i] else ""
        steps_ui.append(html.Div(f"{prefix} {name}{time_str}", style={"color": color, "marginBottom": "5px"}))

    final_btn = html.Button("填寫意見回饋", id="to-comment") if progress['done'] and not progress['error'] else ""

    # 當 done 或 error 就關掉 interval
    stop_interval = progress['done'] or progress['error']

    return steps_ui, final_btn, stop_interval


if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
@app.callback(
    Output("step-list", "children"),
    Output("progress-label", "children"),
    Output("interval", "disabled"),  # <== 多回傳這個
    Input("interval", "n_intervals"),
)
def update_ui(n):
    current = progress['current_index']
    step_status = progress['step_status']
    step_times = progress['step_times']
    steps_total = len(step_names)

    # 每步驟的顯示
    step_elements = []
    for i, step in enumerate(step_names):
        status = step_status[i]

        if status == 'pending':
            color = 'gray'
            prefix = "□"
        elif status == 'running':
            color = 'orange'
            prefix = "▶"
        elif status == 'success':
            color = 'green'
            prefix = "✔"
        elif status == 'error':
            color = 'red'
            prefix = "❌"
        else:
            color = 'gray'
            prefix = "□"

        time_spent = f" ({step_times[i]}秒)" if step_times[i] is not None else ""
        step_elements.append(html.Div(f"{prefix} {step}{time_spent}", style={"color": color, "marginBottom": "5px"}))

    # 顯示文字
    if progress['error']:
        label = f"流程中斷：{step_names[current]} 發生錯誤"
        stop_interval = True
    elif all(status == 'success' for status in step_status):
        label = "完成所有步驟！"
        stop_interval = True
    else:
        label = f"目前進度：{current + 1}/{steps_total}"
        stop_interval = False

    return step_elements, label, stop_interval
